In [69]:
# Basic Prompt Text Summarization
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import  ChatPromptTemplate
from langchain_core.output_parsers import  StrOutputParser

_ = load_dotenv()

speech = """
Today, I stand before you not just as the Prime Minister of Pakistan but as a 
representative of billions of people whose voices are often unheard. The world 
faces grave challenges—climate change, global inequality, and the erosion of human 
rights. In South Asia, peace remains elusive due to unresolved conflicts, particularly 
in Jammu and Kashmir, where people have been denied their right to self-determination. 
I urge this assembly to uphold justice and the principles of the United Nations Charter. 
Let us come together to combat injustice, protect our planet, and create a fairer world for future generations.
"""
llm = ChatOpenAI(model="gpt-4o-mini")

In [70]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant to summarize text."),
    HumanMessage(content=f"Please provide a short concise summary of the following speech: \nTEXT: {speech}"),
])


response = llm.invoke(prompt.format_messages())
response.content

'In his speech, the Prime Minister of Pakistan emphasizes his role as a voice for the marginalized and highlights critical global issues such as climate change, inequality, and human rights violations. He calls for collective action to address conflicts in South Asia, specifically in Jammu and Kashmir, advocating for justice and adherence to the United Nations Charter to foster a more equitable and sustainable future.'

In [71]:
# Prompt Template Summarization
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following speech: \nTEXT: {speech}
    & translate the summary into {language} language
    """),
])

chain = prompt | llm

response = chain.invoke({"speech": speech, "language": "Turkish"})
response.content

"**Summary:**\nThe Prime Minister of Pakistan speaks on behalf of marginalized voices worldwide, addressing significant global issues such as climate change, inequality, and human rights violations. He highlights the ongoing conflicts in South Asia, particularly in Jammu and Kashmir, and calls for the assembly to uphold justice and the UN Charter to foster a just and sustainable future.\n\n**Turkish Translation:**\nPakistan Başbakanı, marjinalleşmiş seslerin temsilcisi olarak, iklim değişikliği, eşitsizlik ve insan hakları ihlalleri gibi önemli küresel sorunlara değiniyor. Özellikle Jammu ve Keşmir'deki devam eden çatışmalara dikkat çekiyor ve meclisi, adaleti savunmaya ve BM Şartı'na bağlı kalmaya çağırarak adil ve sürdürülebilir bir gelecek oluşturma çağrısında bulunuyor."

In [72]:
# Stuff Documentation Chain:
from pypdf import PdfReader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import ChatPromptTemplate

# Load the PDF content
pdfReader = PdfReader("Proposal.pdf")
text = ""

for i, page in enumerate(pdfReader.pages):
    content = page.extract_text()
    if content:
        text += content

# Create a Document object
document = [Document(page_content=text)]

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following document: \nTEXT: {docs}
    & translate into {language} language
    """),
])

# Load the summarization chain, ensuring document_variable_name matches the input variable in the prompt
chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    prompt=prompt,
    document_variable_name="docs",  # Ensure this matches the input variable in the prompt
    verbose=False
)

# Invoke the chain with the document
response = chain.invoke({"input_documents": document, "language" : "Turkish"})

# Print the response
print(response["output_text"])


**Summary:**

The letter is an expression of admiration and evolving feelings from the sender towards the recipient, who is dedicated to her studies in applied physics. The sender praises her achievements and acknowledges the complexities of their relationship due to distance and personal commitments. They convey respect for her goals and express a desire for open communication, whether it leads to a deeper connection or remains a strong friendship. The sender emphasizes their support for her choices and hopes for future discussions, wishing her happiness and success regardless of the outcome.

---

**Turkish Translation:**

**Özet:**

Mektup, gönderenin alıcıya karşı duyduğu hayranlık ve gelişen hisleri ifade etmektedir. Alıcının uygulamalı fizik alanındaki çalışmalarına olan bağlılığı övülmekte ve onun başarıları takdir edilmektedir. Gönderen, aralarındaki mesafe ve kişisel taahhütler nedeniyle ilişkilerinin karmaşıklığını kabul eder. Alıcının hedeflerine saygı göstererek açık bir il

In [73]:
# Map Reduce Summarization:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import ChatPromptTemplate

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.create_documents(texts=[text])

# Define the map prompt
map_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following text:
    TEXT: {text}
    Also, translate the summary into {language}.
    """),
])

# Define the reduce prompt
reduce_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Combine the following summaries into a single concise summary:
    TEXT: {text}
    Also, translate the combined summary into {language}.
    """),
])

# Load the summarization chain
chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=reduce_prompt,
    verbose=False,
)

# Prepare inputs for the chain
inputs = {
    "input_documents": chunks,
    "language": "Turkish"
}

# Invoke the chain with the inputs
response = chain.invoke(inputs)

# Print the output summary
print(response["output_text"])


**Combined Summary:**  
The writer expresses admiration for the recipient's dedication and achievements in applied physics while acknowledging their deepening feelings for them. They emphasize the importance of open communication and respect for the recipient's goals and choices, highlighting their intelligence and character. The writer conveys unwavering support for the recipient's happiness and comfort, valuing family bonds above all. They invite the recipient to share their thoughts and conclude with well wishes for their success.

**Turkish Translation:**  
Yazar, alıcının uygulamalı fizik alanındaki özverisi ve başarıları için hayranlık duyduğunu ifade ederken, onlara karşı duygularının derinleştiğini kabul ediyor. Açık iletişimin ve alıcının hedeflerine ve seçimlerine saygının önemini vurguluyor, zekalarını ve karakterlerini öne çıkarıyor. Yazar, alıcının mutluluğu ve rahatlığı için sarsılmaz bir destek sunduğunu, aile bağlarını her şeyin üzerinde değerlendirdiğini belirtiyor. Al

In [74]:
# Refine Summarization:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.create_documents(texts=[text])

# Define the map prompt
map_prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
    You are a helpful assistant. Your task is to summarize and translate the given text.
    1. Summarize the following text into a short and concise summary.
    2. Translate the summary into {language}.
    TEXT: {text}
    """
)

# Define the reduce (refine) prompt
reduce_prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
    You are a helpful assistant. Your task is to combine summaries and translate them.
    1. Combine the following summaries into one concise and coherent summary.
    2. Translate the final summary into {language}.
    TEXT: {text}
    """
)

# Create the LLM chains for map and refine steps
map_chain = LLMChain(llm=llm, prompt=map_prompt)
refine_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Create the RefineDocumentsChain
chain = RefineDocumentsChain(
    initial_llm_chain=map_chain,
    refine_llm_chain=refine_chain,
    document_variable_name="text",
    initial_response_name="output_text",
    verbose=True,
)

# Prepare inputs for the chain
inputs = {
    "input_documents": chunks,
    "language": "Turkish"
}

# Run the chain
response = chain(inputs)

# Print the output summary
print(response["output_text"])




> Entering new RefineDocumentsChain chain...

> Finished chain.
**Combined Summary:**

The writer expresses the importance of family bonds and emphasizes their desire to maintain a strong relationship without jeopardizing it. They hope the letter encourages further dialogue and understanding, inviting the recipient to share any thoughts or concerns. The writer values the recipient's feelings and extends best wishes for their happiness, success, and fulfillment in life.

**Turkish Translation:**

Yazarı, aile bağlarının önemini vurguluyor ve güçlü bir ilişkiyi sürdürme arzusunu ifade ediyor. Mektubun daha fazla diyalog ve anlayış teşvik etmesini umuyor ve alıcıyı düşüncelerini veya endişelerini paylaşmaya davet ediyor. Yazar, alıcının duygularını önemsiyor ve hayatında mutluluk, başarı ve tatmin dileklerini iletiyor.
